In [4]:
import os
import shutil  # модуль для работы с файлами (копирование, перемещение)
import json  # модуль для работы с JSON
from PIL import Image  # библиотека для работы с изображениями

# Директории для изображений и аннотаций
train_dir = "./test/images"  # папка для хранения изображений
train_labels_dir = "./test/labels"  # папка для хранения аннотаций
os.makedirs(train_dir, exist_ok=True)  # создаем папку, если не существует
os.makedirs(train_labels_dir, exist_ok=True)  # аналогично для папки аннотаций

# Функция нормализации координат в формат YOLO (координаты центра и размеры в долях от ширины и высоты изображения)
def normalize_yolo(x, y, w, h, img_width, img_height):
    x_center = (x + w / 2) / img_width  # координата центра по оси x
    y_center = (y + h / 2) / img_height  # координата центра по оси y
    width = w / img_width  # ширина объекта относительно ширины изображения
    height = h / img_height  # высота объекта относительно высоты изображения
    return x_center, y_center, width, height  # возвращаем нормализованные значения

# Функция для изменения размера изображений и аннотаций под новые размеры
def resize_image_and_labels(image_path, label_file, target_size=(640, 384)):
    with Image.open(image_path) as img:  # открываем изображение
        original_size = img.size  # исходный размер изображения (ширина, высота)
        img_resized = img.resize(target_size)  # изменяем размер изображения на указанный
        img_resized.save(image_path)  # сохраняем изображение с новым размером

    # Если файл аннотации существует
    if os.path.exists(label_file):
        with open(label_file, 'r') as f:
            annotations = f.readlines()  # читаем все строки из файла аннотаций

        new_annotations = []
        # Проходим по каждой аннотации
        for annotation in annotations:
            parts = annotation.strip().split()  # разбиваем строку на части
            class_id = parts[0]  # идентификатор класса (например, мяч)
            x_center, y_center, width, height = map(float, parts[1:])  # берем координаты и размеры

            # Пересчет координат и размеров для нового размера изображения
            x_center = x_center * target_size[0] / original_size[0]  # масштабируем координату x
            y_center = y_center * target_size[1] / original_size[1]  # масштабируем координату y
            width = width * target_size[0] / original_size[0]  # масштабируем ширину
            height = height * target_size[1] / original_size[1]  # масштабируем высоту

            # Формируем новую строку аннотации
            new_annotations.append(f"{class_id} {x_center} {y_center} {width} {height}")

        # Записываем новые аннотации в файл
        with open(label_file, 'w') as f:
            f.write('\n'.join(new_annotations))  # записываем обновленные аннотации

# Функция обработки мяча из файла JSON (аннотации мяча)
def process_ball_from_ljson(ljson_file, img_width, img_height, label_out_dir, img_file):
    label_data = {}  # словарь для хранения данных аннотаций

    frame_id = int(os.path.splitext(img_file)[0])  # берем номер кадра из имени файла изображения

    # Читаем JSON файл
    with open(ljson_file, 'r') as f:
        data = json.load(f)  # загружаем данные JSON

        # Проходим по каждой аннотации в JSON
        for item in data['annotations']:
            if isinstance(item, dict) and item.get("attributes", {}).get("role") == "ball":  # проверяем, что это мяч
                annotation_frame_id = int(item['image_id'][-3:])  # берем последние три цифры id кадра

                if annotation_frame_id == frame_id:  # если id кадра совпадает с изображением
                    bbox = item.get("bbox_image", {})  # берем bounding box (координаты) объекта
                    x, y, w, h = bbox.get("x", 0), bbox.get("y", 0), bbox.get("w", 0), bbox.get("h", 0)
                    # Нормализуем координаты в формат YOLO
                    x_center, y_center, width, height = normalize_yolo(x, y, w, h, img_width, img_height)

                    # Добавляем аннотацию в словарь
                    if img_file not in label_data:
                        label_data[img_file] = []
                    label_data[img_file].append(f"0 {x_center} {y_center} {width} {height}")  # класс 0 (мяч)

    # Если есть аннотации для текущего изображения
    if img_file in label_data:
        # Записываем их в файл
        label_file = os.path.join(label_out_dir, f"{os.path.splitext(img_file)[0]}.txt")
        with open(label_file, 'w') as f:
            f.write('\n'.join(label_data[img_file]))  # записываем аннотацию

# Функция копирования изображений и аннотаций для мяча
def copy_images_and_labels_for_ball(sequence_dir, img_width, img_height, dest_img_dir, dest_label_dir, ljson_file):
    img_dir = os.path.join(sequence_dir, "img1")  # папка с изображениями

    # Директория для хранения картинок
    sequence_name = os.path.basename(sequence_dir)  # имя последовательности (например, SNGS-123)
    sequence_img_dir = os.path.join(dest_img_dir, sequence_name)  # путь для сохранения изображений
    os.makedirs(sequence_img_dir, exist_ok=True)  # создаем директорию для картинок

    # Директория для хранения аннотаций
    sequence_label_dir = os.path.join(dest_label_dir, sequence_name)  # путь для сохранения аннотаций
    os.makedirs(sequence_label_dir, exist_ok=True)  # создаем директорию для аннотаций

    # Проходим по всем изображениям в папке
    for img_file in os.listdir(img_dir):
        if img_file.endswith(".jpg"):  # если файл является изображением
            img_path = os.path.join(img_dir, img_file)  # полный путь к изображению

            # Новый путь для сохранения изображения
            new_img_path = os.path.join(sequence_img_dir, img_file)
            shutil.copy(img_path, new_img_path)  # копируем изображение

            # Путь для аннотации
            label_file = os.path.join(sequence_label_dir, f"{os.path.splitext(img_file)[0]}.txt")
            resize_image_and_labels(new_img_path, label_file)  # изменяем размер изображения и аннотаций

            # Обрабатываем JSON файл для аннотации мяча
            process_ball_from_ljson(ljson_file, img_width, img_height, sequence_label_dir, img_file)

# Основная часть кода, обрабатывающая все последовательности
base_train_dir = "./data/SoccerNetGS/test/"  # директория с последовательностями
sequences = [d for d in os.listdir(base_train_dir) if d.startswith("SNGS-")]  # список всех последовательностей

# Проходим по каждой последовательности
for sequence in sequences:
    sequence_dir = os.path.join(base_train_dir, sequence)  # путь к каждой последовательности
    ljson_file = os.path.join(sequence_dir, "Labels-GameState.json")  # путь к JSON файлу аннотаций
    img_width = 1920  # исходная ширина изображения
    img_height = 1080  # исходная высота изображения
    # Копируем изображения и аннотации для текущей последовательности
    copy_images_and_labels_for_ball(sequence_dir, img_width, img_height, train_dir, train_labels_dir, ljson_file)
